In [ ]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

In [ ]:
curl 172.29.211.112:11434/api/chat -d '{
  "model": "llama3-chatqa:8b-v1.5-q5_K_M",
  "messages": [
    {
      "role": "user",
      "content": "why is the sky blue?"
    }
  ],
  "stream": true
}'

In [ ]:
http://172.29.208.1

http://172.17.0.0

http://172.17.0.1

In [ ]:
[Unit]
Description=Ollama Service
After=network-online.target

[Service]
ExecStart=/home/jens/ollama/ollama serve
User=ollama
Group=ollama
Restart=always
RestartSec=3
Environment="OLLAMA_FLASH_ATTENTION=1"
Environment="OLLAMA_HOST=0.0.0.0:11434"
Environment="HOME=/home/jens/ollama"
Environment="PATH=/home/linuxbrew/.linuxbrew/bin:/home/linuxbrew/.linuxbrew/>
Environment="OLLAMA_MAX_VRAM=8589934592"
Environment="OLLAMA_FLASH_ATTENTION=1"
Environment="OLLAMA_KEEP_ALIVE=-1"
Environment="OLLAMA_ORIGINS=https://172.17.0.1,http://172.17.0.1,https://172>

[Install]
WantedBy=multi-user.target

In [ ]:
loader = PDFMinerPDFasHTMLLoader('D:/school/ALL_PDFs/well_structured/A20080047930.pdf')

In [ ]:
data = loader.load()[0]   # entire PDF is loaded as a single Document

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('span')

In [ ]:
content

In [ ]:
import re
cur_fsm = None
cur_fs = None
cur_text = ''
snippets = []   # first collect all snippets that have the same font size
for c in content:
    st = c.get('style')
    if not st:
        continue
    fsm = re.findall('font-family:(.*?);',st)
    fs = re.findall('font-size:(\d+)px',st)
    if not fsm or not fs:
        continue
    fsm = str(fsm[0]).strip()
    fs = int(fs[0])
    if not cur_fsm: 
        cur_fsm = fsm
    if not cur_fs:
        cur_fs = fs      
    if fsm == cur_fsm and fs == cur_fs: 
        cur_text += c.text  
    else:
        snippets.append((cur_text,cur_fsm,cur_fs))
        cur_fsm = fsm
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text,cur_fsm,cur_fs))

In [ ]:
tempsnippets = snippets
snippets = []
for i in range(len(tempsnippets)):
    if i >= 4:
        snippets.append(tempsnippets[i])

In [ ]:
snippets

In [ ]:
for snip in snippets:
    if snip[1] != 'TimesNewRomanPS-BoldMT':
        print(snip[1])

In [ ]:
for snip in snippets: 
    print(snip[1])

In [ ]:
for snip in snippets:
    if len(snip[0]) > 100:
        print('Text:',snip[0])
    

In [ ]:
for snip in snippets :
    print('Font size:',snip[1])
    print('Text:',snip[0])
    print('---')
    print()

In [ ]:
for snip in snippets:
    if snip[1] == 8:
        print('Text:',snip[0])    

In [ ]:
from langchain_community.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    # if current snippet's font size > previous section's heading => it is a new heading
    if not semantic_snippets  or s[2] > semantic_snippets[cur_idx].metadata['heading_fontS']:   
        metadata={'heading':s[0],'content_fontM':'', 'heading_fontM':s[1], 'content_fontS': 0, 'heading_fontS': s[2]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue
    
    if s[1] != semantic_snippets[cur_idx].metadata['content_fontM']:
        if semantic_snippets[cur_idx].page_content != '':
                if 'Bold' in s[1] or 'Italic' in s[1] or 'BoldItalic' in s[1] or 'BoldOblique' in s[1] or 'BoldOblique' in s[1]:
                    metadata={'heading':s[0],'content_fontM':'', 'heading_fontM':s[1], 'content_fontS': 0, 'heading_fontS': s[2]}
                    metadata.update(data.metadata)
                    semantic_snippets.append(Document(page_content='',metadata=metadata))
                    cur_idx += 1
                    continue
    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)

        
    if not semantic_snippets[cur_idx].metadata['content_fontS'] or not semantic_snippets[cur_idx].metadata['content_fontM'] or not 'Bold' in s[1] or not 'Italic' in s[1] or not 'BoldItalic' in s[1] or not 'BoldOblique' in s[1] or not  'BoldOblique' in s[1] or s[2] <= semantic_snippets[cur_idx].metadata['content_fontS']:
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_fontM'] = s[1] if not semantic_snippets[cur_idx].metadata['content_fontM'] else semantic_snippets[cur_idx].metadata['content_fontM']
        semantic_snippets[cur_idx].metadata['content_fontS'] = max(s[2], semantic_snippets[cur_idx].metadata['content_fontS'])
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    
  
    
    if s[2] > semantic_snippets[cur_idx].metadata['content_fontS'] and s[2] < semantic_snippets[cur_idx].metadata['heading_fontS']:
        metadata={'heading':s[0],'content_fontM':'', 'heading_fontM':s[1], 'content_fontS': 0, 'heading_fontS': s[2]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue
    
    
    
    metadata={'heading':s[0], 'content_fontM':'', 'heading_fontM':s[1],'content_fontS': 0, 'heading_fontS': s[2]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

In [ ]:
string1 = 'hello friend'
string2 = 'hello friend how are you'

if 'ell' in string2:
    print('yes')

In [ ]:
for snip in semantic_snippets:
    print('HEADING:  ', snip.metadata['heading'], snip.metadata['heading_fontM'])
    print('-----------------')
    print()
    print('CONTENT LEN:  ',len(snip.page_content))
    print('---------------------------------------------------------------------------------')
    print()


In [ ]:


for snip in semantic_snippets:
    print('HEADING:  ',snip.metadata['heading'])
    print('-----------------')
    print()
    print('CONTENT:  ',snip.page_content)
    print('---------------------------------------------------------------------------------')
    print()


In [ ]:
semantic_snippets

In [ ]:
semantic_snippets[0].metadata['content_font']

In [ ]:
for each in semantic_snippets:
    if len(each.page_content) > 130:
        print('Heading:',each.metadata['heading'])
        print('Content:',each.page_content)
        print('---')
        print()